In [1]:
!pip install transformers datasets accelerate -U

     ------------------------------------ 138.0/138.0 kB 327.5 kB/s eta 0:00:00
   ---------------------------------------- 9.0/9.0 MB 75.4 kB/s eta 0:00:00
   --------------------------------------- 542.0/542.0 kB 92.5 kB/s eta 0:00:00
   -------------------------------------- 302.4/302.4 kB 121.4 kB/s eta 0:00:00
   --------------------------------------- 401.2/401.2 kB 93.0 kB/s eta 0:00:00
   --------------------------------------- 133.4/133.4 kB 91.6 kB/s eta 0:00:00
   --------------------------------------- 116.3/116.3 kB 74.6 kB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.4
    Uninstalling dill-0.3.4:
      Successfully uninstalled dill-0.3.4
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.1
    Uninstalling huggingface-hub-0.20.1:
      Successfully uninstalled huggingface-hub-0.20.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.3
    Uninstalling accel

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
spyder 5.2.2 requires ipython<8.0.0,>=7.6.0, but you have ipython 8.18.1 which is incompatible.


In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Load your data
data = pd.read_csv('fashion_products.csv')

# Prepare input_text by concatenating multiple columns
data['input_text'] = data.apply(lambda row: f"Describe this product: Product Name {row['Product Name']}, {row['Brand']} Brand, {row['Category']} Category, {row['Color']} Color, {row['Size']} Size .", axis=1)
data['target_text'] = data['Price'].apply(lambda x: f"The estimated price is {x}")

# Split data
train_data, val_data = train_test_split(data, test_size=0.1)
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("MBZUAI/LaMini-Flan-T5-248M")

def tokenize_function(examples):
    model_inputs = tokenizer(examples['input_text'], max_length=512, truncation=True, padding="max_length")
    model_inputs['labels'] = tokenizer(examples['target_text'], max_length=100, truncation=True, padding="max_length").input_ids
    return model_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

C:\Users\muham\Downloads\Music\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer, AutoModelForSeq2SeqLM

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',          # where to save the model
    evaluation_strategy="epoch",     # evaluation is done at the end of each epoch
    learning_rate=5e-5,              # learning rate
    per_device_train_batch_size=4,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    num_train_epochs=1,              # number of training epochs
    weight_decay=0.01                # strength of weight decay
)

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained("MBZUAI/LaMini-Flan-T5-248M")

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,     # your training dataset
    eval_dataset=val_dataset,        # your validation dataset
    tokenizer=tokenizer,             # your tokenizer
)

# Start training
trainer.train()

C:\Users\muham\Downloads\Music\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\muham\Downloads\Music\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
model.save_pretrained('product_model')
tokenizer.save_pretrained('product_model')